In [ ]:
pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 8.6 MB/s eta 0:00:00


In [ ]:
from torch import nn
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch import autograd
import numpy as np
from torchmetrics.functional import r2_score
import matplotlib.pyplot as plt
from google.colab import files
from numpy import inf

In [ ]:
!mkdir images

In [ ]:
class MyDataset(Dataset):
 
  def __init__(self,file_name):
    df_doped=pd.read_csv(file_name)
    for i in range(1, 321):
      df_doped.iloc[:,i] = (df_doped.iloc[:,i] - df_doped.iloc[:,i].min()) / (df_doped.iloc[:,i].max() - df_doped.iloc[:,i].min())

 
    x=df_doped.iloc[:,0:641].values
    y=df_doped.iloc[:,961:1281].values

    for i in range(0,1):
      bias = x[:,i]

      bias_inv = np.reciprocal(bias)
      bias_inv[bias_inv == inf] = 0

      bias_0_5 = np.power(bias,0.5)
      bias_inv_0_5 = np.power(bias_inv,0.5)

      bias_2 = np.power(bias,2)
      bias_inv_2 = np.power(bias_inv,2)

      bias_3 = np.power(bias,3)
      bias_inv_3 = np.power(bias_inv,3)

      bias_log = np.log(bias)
      bias_log_inv = np.reciprocal(bias_log)
      bias_log[bias_log==-inf] = 0
      bias_log[bias_log==inf] = 0 
      bias_log_inv[bias_log==-inf] = 0
      bias_log_inv[bias_log==inf] = 0 

      bias_neg = bias*(-1)
      bias_exp = np.exp(bias)
      bias_neg_exp = np.exp(bias_neg)

      b = bias_inv.reshape(-1,1)
      x = np.append(x,b,axis=1)

      b = bias_0_5.reshape(-1,1)
      x = np.append(x,b,axis=1)


      b = bias_inv_0_5.reshape(-1,1)
      x = np.append(x,b,axis=1)

      b = bias_2.reshape(-1,1)
      x = np.append(x,b,axis=1)

      b = bias_inv_2.reshape(-1,1)
      x = np.append(x,b,axis=1)

      b = bias_3.reshape(-1,1)
      x = np.append(x,b,axis=1)

      b = bias_inv_3.reshape(-1,1)
      x = np.append(x,b,axis=1)

      b = bias_log.reshape(-1,1)
      x = np.append(x,b,axis=1)

      b = bias_log_inv.reshape(-1,1)
      x = np.append(x,b,axis=1)

      b = bias_exp.reshape(-1,1)
      x = np.append(x,b,axis=1)

      b = bias_neg_exp.reshape(-1,1)
      x = np.append(x,b,axis=1)
    
 
    self.x_train=torch.tensor(x,dtype=torch.float32)
    self.y_train=torch.tensor(y,dtype=torch.float32)
 
  def __len__(self):
    return len(self.y_train)
   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]

In [ ]:
myDs=MyDataset("Real_Train_cleaned.csv")
train_loader=DataLoader(myDs,batch_size=10,shuffle=True)

<ipython-input-3-352dc9e63123>:15: RuntimeWarning: divide by zero encountered in reciprocal
  bias_inv = np.reciprocal(bias)
<ipython-input-3-352dc9e63123>:27: RuntimeWarning: divide by zero encountered in log
  bias_log = np.log(bias)


In [ ]:
for i, (data, labels) in enumerate(train_loader):
  print(data.shape, labels.shape)
  print(data,labels)
  break;

torch.Size([10, 652]) torch.Size([10, 320])
tensor([[ 0.0500,  0.4444,  0.4444,  ..., -0.3338,  1.0513,  0.9512],
        [ 0.4800,  0.4444,  0.4444,  ..., -1.3625,  1.6161,  0.6188],
        [ 0.5000,  0.0000,  0.0000,  ..., -1.4427,  1.6487,  0.6065],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ..., -0.0000,  1.0000,  1.0000],
        [ 0.6500,  1.0000,  1.0000,  ..., -2.3214,  1.9155,  0.5220],
        [ 0.1000,  0.4444,  0.4444,  ..., -0.4343,  1.1052,  0.9048]]) tensor([[0.1997, 0.1997, 0.1997,  ..., 0.1505, 0.1505, 0.1505],
        [0.6297, 0.6297, 0.6297,  ..., 0.1505, 0.1505, 0.1505],
        [0.5362, 0.5362, 0.5362,  ..., 0.0400, 0.0400, 0.0400],
        ...,
        [0.0369, 0.0369, 0.0369,  ..., 0.0398, 0.0398, 0.0398],
        [0.8921, 0.8921, 0.8921,  ..., 0.2426, 0.2426, 0.2426],
        [0.2497, 0.2497, 0.2497,  ..., 0.1505, 0.1505, 0.1505]])


In [ ]:
"""

##Check for NaN values
model = nn.Sequential(nn.Linear(321, 960),
                      nn.ReLU(),
                      nn.Linear(960, 640),
                      nn.ReLU(),
                      nn.Linear(640, 320),
                      nn.Sigmoid())

# Define the loss
criterion = nn.MSELoss()
# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.Adam(model.parameters(), lr=0.0005)
epochs = 5
for e in range(epochs):
    running_loss = 0
    for data, labels in train_loader:
      with autograd.detect_anomaly():
          # Training pass
          optimizer.zero_grad()    
          output = model(data)
          loss = criterion(output, labels)
          print("The MSE Loss is: ",loss.item(),'\n')
          loss.backward()
          optimizer.step()
  """

In [ ]:
model = nn.Sequential(nn.Linear(652, 512),
                      nn.ReLU(),
                      nn.Linear(512, 512),
                      nn.ReLU(),
                      nn.Linear(512, 320))

# Define the loss
criterion = nn.MSELoss()
MAE = nn.L1Loss()
# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.Adam(model.parameters(), lr=0.0005)
epochs = 55
for e in range(epochs):
    running_loss = 0
    r_score = 0
    for data, labels in train_loader:
        # Training pass
        optimizer.zero_grad()    
        output = model(data)
        loss = criterion(output, labels)
        output_mae = MAE(output, labels)
        print("The MAE Loss is: ",output_mae.item(),'\n')
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        ##r_score = torch.sum(r2score)/r2score.size(dim=0)
    else:
        print(f"Training loss: {running_loss/len(train_loader)}, R score: {r2_score(output, labels, multioutput='variance_weighted')}")
        if (running_loss/len(train_loader) < 0.0015) and (r2_score(output, labels, multioutput='variance_weighted')>0.95):
          break


Streaming output truncated to the last 5000 lines.
The MAE Loss is:  0.06513891369104385 

The MAE Loss is:  0.0663321390748024 

The MAE Loss is:  0.04381486028432846 

The MAE Loss is:  0.05026105418801308 

The MAE Loss is:  0.08521866798400879 

The MAE Loss is:  0.05615299195051193 

The MAE Loss is:  0.058519937098026276 

The MAE Loss is:  0.0904332846403122 

The MAE Loss is:  0.056431226432323456 

The MAE Loss is:  0.06866186112165451 

The MAE Loss is:  0.10532266646623611 

The MAE Loss is:  0.07713610678911209 

The MAE Loss is:  0.045405760407447815 

The MAE Loss is:  0.06346917152404785 

The MAE Loss is:  0.075630322098732 

The MAE Loss is:  0.0742521733045578 

The MAE Loss is:  0.07735658437013626 

The MAE Loss is:  0.050896234810352325 

The MAE Loss is:  0.06519146263599396 

The MAE Loss is:  0.075100377202034 

The MAE Loss is:  0.07037555426359177 

The MAE Loss is:  0.07870316505432129 

The MAE Loss is:  0.04664153233170509 

The MAE Loss is:  0.065417356789

In [ ]:
df_test = pd.read_csv("Real_Test.csv")

for i in range(1, 321):
  df_test.iloc[:,i] = (df_test.iloc[:,i] - df_test.iloc[:,i].min()) / (df_test.iloc[:,i].max() - df_test.iloc[:,i].min())

x = df_test.iloc[:,0:641].values
y = df_test.iloc[:,961:1281].values
for i in range(0,1):
  bias = x[:,i]

  bias_inv = np.reciprocal(bias)
  bias_inv[bias_inv == inf] = 0

  bias_0_5 = np.power(bias,0.5)
  bias_inv_0_5 = np.power(bias_inv,0.5)

  bias_2 = np.power(bias,2)
  bias_inv_2 = np.power(bias_inv,2)

  bias_3 = np.power(bias,3)
  bias_inv_3 = np.power(bias_inv,3)

  bias_log = np.log(bias)
  bias_log_inv = np.reciprocal(bias_log)
  bias_log[bias_log==-inf] = 0
  bias_log[bias_log==inf] = 0 
  bias_log_inv[bias_log==-inf] = 0
  bias_log_inv[bias_log==inf] = 0 

  bias_neg = bias*(-1)
  bias_exp = np.exp(bias)
  bias_neg_exp = np.exp(bias_neg)

  b = bias_inv.reshape(-1,1)
  x = np.append(x,b,axis=1)

  b = bias_0_5.reshape(-1,1)
  x = np.append(x,b,axis=1)


  b = bias_inv_0_5.reshape(-1,1)
  x = np.append(x,b,axis=1)

  b = bias_2.reshape(-1,1)
  x = np.append(x,b,axis=1)

  b = bias_inv_2.reshape(-1,1)
  x = np.append(x,b,axis=1)

  b = bias_3.reshape(-1,1)
  x = np.append(x,b,axis=1)

  b = bias_inv_3.reshape(-1,1)
  x = np.append(x,b,axis=1)

  b = bias_log.reshape(-1,1)
  x = np.append(x,b,axis=1)

  b = bias_log_inv.reshape(-1,1)
  x = np.append(x,b,axis=1)

  b = bias_exp.reshape(-1,1)
  x = np.append(x,b,axis=1)

  b = bias_neg_exp.reshape(-1,1)
  x = np.append(x,b,axis=1)


x_test=torch.tensor(x,dtype=torch.float32)
y_test=torch.tensor(y,dtype=torch.float32)
output_test = model(x_test)
Test_mae = MAE(output_test, y_test)
print("Test mae: ",Test_mae.item(),"\n")

Test mae:  0.026355693116784096 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in reciprocal
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log


In [ ]:
x_flat = df_test.iloc[:,321:641].values
np_output = output_test.cpu().detach().numpy()
x_output = x_flat - np_output
PP = df_test.iloc[:,641:961].values

plt.figure()
for i in range(0, 50):
  plt.plot(x_output[i])
  plt.plot(PP[i])
  plt.xlabel("Samples from contact metal")
  plt.ylabel("Potential")
  plt.savefig("images/output_"+str(i))
  plt.clf()

<Figure size 432x288 with 0 Axes>

In [ ]:
#To download zip
!zip -r /content/images.zip /content/images
files.download("/content/images.zip")

FOR MATLAB TESTING VALUES

In [ ]:
df_test = pd.read_csv("Real_non_converge.csv")

for i in range(1, 321):
  df_test.iloc[:,i] = (df_test.iloc[:,i] - df_test.iloc[:,i].min()) / (df_test.iloc[:,i].max() - df_test.iloc[:,i].min())

x = df_test.iloc[:,0:641].values
y = df_test.iloc[:,961:1281].values
for i in range(0,1):
  bias = x[:,i]

  bias_inv = np.reciprocal(bias)
  bias_inv[bias_inv == inf] = 0

  bias_0_5 = np.power(bias,0.5)
  bias_inv_0_5 = np.power(bias_inv,0.5)

  bias_2 = np.power(bias,2)
  bias_inv_2 = np.power(bias_inv,2)

  bias_3 = np.power(bias,3)
  bias_inv_3 = np.power(bias_inv,3)

  bias_log = np.log(bias)
  bias_log_inv = np.reciprocal(bias_log)
  bias_log[bias_log==-inf] = 0
  bias_log[bias_log==inf] = 0 
  bias_log_inv[bias_log==-inf] = 0
  bias_log_inv[bias_log==inf] = 0 

  bias_neg = bias*(-1)
  bias_exp = np.exp(bias)
  bias_neg_exp = np.exp(bias_neg)

  b = bias_inv.reshape(-1,1)
  x = np.append(x,b,axis=1)

  b = bias_0_5.reshape(-1,1)
  x = np.append(x,b,axis=1)


  b = bias_inv_0_5.reshape(-1,1)
  x = np.append(x,b,axis=1)

  b = bias_2.reshape(-1,1)
  x = np.append(x,b,axis=1)

  b = bias_inv_2.reshape(-1,1)
  x = np.append(x,b,axis=1)

  b = bias_3.reshape(-1,1)
  x = np.append(x,b,axis=1)

  b = bias_inv_3.reshape(-1,1)
  x = np.append(x,b,axis=1)

  b = bias_log.reshape(-1,1)
  x = np.append(x,b,axis=1)

  b = bias_log_inv.reshape(-1,1)
  x = np.append(x,b,axis=1)

  b = bias_exp.reshape(-1,1)
  x = np.append(x,b,axis=1)

  b = bias_neg_exp.reshape(-1,1)
  x = np.append(x,b,axis=1)


x_test=torch.tensor(x,dtype=torch.float32)
output_test = model(x_test)
output_test

<ipython-input-7-f84e208e6394>:11: RuntimeWarning: divide by zero encountered in reciprocal
  bias_inv = np.reciprocal(bias)
<ipython-input-7-f84e208e6394>:23: RuntimeWarning: divide by zero encountered in log
  bias_log = np.log(bias)


tensor([[0.2435, 0.2805, 0.2850,  ..., 0.2890, 0.2555, 0.2734],
        [0.2513, 0.2745, 0.2969,  ..., 0.2865, 0.2536, 0.2710],
        [0.4647, 0.4752, 0.4774,  ..., 0.0551, 0.0168, 0.0141],
        [0.4344, 0.4344, 0.4649,  ..., 0.0365, 0.0382, 0.0196],
        [0.3351, 0.3523, 0.3412,  ..., 0.0607, 0.0410, 0.0368],
        [0.4588, 0.4704, 0.4674,  ..., 0.1577, 0.1555, 0.1161]],
       grad_fn=<AddmmBackward0>)

In [ ]:
np_output = output_test.cpu().detach().numpy()
np_input = (-1)*np_output
np_input

array([[-0.24351133, -0.2805081 , -0.28495386, ..., -0.2889949 ,
        -0.25548083, -0.27336577],
       [-0.2513055 , -0.27449745, -0.29691887, ..., -0.2865447 ,
        -0.25359717, -0.2710481 ],
       [-0.46473926, -0.4752359 , -0.4774018 , ..., -0.05509324,
        -0.0167619 , -0.01406523],
       [-0.4343922 , -0.43441814, -0.46485025, ..., -0.03645164,
        -0.03820053, -0.01960689],
       [-0.33509326, -0.35226357, -0.3412345 , ..., -0.0607101 ,
        -0.04098717, -0.03675717],
       [-0.45877382, -0.470411  , -0.4674135 , ..., -0.1577411 ,
        -0.15551558, -0.11609258]], dtype=float32)

In [ ]:
U_guess = pd.DataFrame(np_input)
U_guess.to_csv('U_guess.csv')

FOR FORMATTING VALUES FOR MATLAB

In [ ]:
import re
re.sub(r'\s+', ';', '-0.45877382	-0.470411	-0.4674135	-0.4736074	-0.44156438	-0.47548252	-0.44735974	-0.44824904	-0.4639969	-0.45784265	-0.45644265	-0.4638867	-0.46670336	-0.46304768	-0.48638973	-0.4515526	-0.4706145	-0.49355316	-0.4642372	-0.4766898	-0.46823722	-0.45529616	-0.4491247	-0.48889023	-0.4732201	-0.4671613	-0.4744037	-0.45908988	-0.49704504	-0.4645567	-0.47549337	-0.4682126	-0.43353945	-0.46368045	-0.44632033	-0.4935618	-0.45706254	-0.4611628	-0.46721405	-0.46532038	-0.45529568	-0.4515638	-0.43584064	-0.46726593	-0.45786634	-0.46373707	-0.45947528	-0.47260565	-0.4767009	-0.4698779	-0.4660646	-0.4894919	-0.46707913	-0.46301925	-0.4391646	-0.4473611	-0.4579038	-0.43909606	-0.46308517	-0.4789041	-0.47968763	-0.4640072	-0.46033573	-0.44862735	-0.4606386	-0.49757892	-0.48357502	-0.4733503	-0.45197883	-0.45779502	-0.4625774	-0.473577	-0.45132506	-0.46539423	-0.48512572	-0.44213033	-0.4576674	-0.44851744	-0.45456725	-0.45654044	-0.43471223	-0.47352314	-0.4519704	-0.47006404	-0.4716673	-0.48033005	-0.4691529	-0.44905692	-0.4571511	-0.46508205	-0.47522745	-0.44258773	-0.43368834	-0.4387396	-0.4314227	-0.451791	-0.4546328	-0.44078672	-0.46844348	-0.47911972	-0.42407608	-0.4820208	-0.4335001	-0.4391842	-0.42875797	-0.45055637	-0.41232306	-0.46343625	-0.45982605	-0.4398269	-0.46870744	-0.46729568	-0.4749689	-0.43989575	-0.4789515	-0.48303896	-0.4574746	-0.44422472	-0.47013706	-0.46810663	-0.46371156	-0.4725048	-0.4183119	-0.41138417	-0.4178686	-0.4214002	-0.412555	-0.40876997	-0.44449925	-0.40183288	-0.41817838	-0.4190626	-0.45148662	-0.4255525	-0.40971616	-0.4155901	-0.4407776	-0.43564308	-0.42403418	-0.42493048	-0.400456	-0.40634727	-0.41420576	-0.36746252	-0.34376648	-0.40012938	-0.32951978	-0.30678058	-0.32473597	-0.30311936	-0.30066863	-0.27692956	-0.2479739	-0.24635434	-0.26435274	-0.24624598	-0.26581013	-0.2772556	-0.28556728	-0.2832247	-0.2942627	-0.26503024	-0.3463428	-0.2982153	-0.29151022	-0.3057433	-0.29632676	-0.3076282	-0.31120262	-0.3089162	-0.2807099	-0.30578163	-0.30487075	-0.3019795	-0.30392623	-0.31249702	-0.27721888	-0.27108496	-0.2798808	-0.21898511	-0.2596537	-0.2405209	-0.21948177	-0.21768528	-0.18721965	-0.17497492	-0.15110993	-0.16486979	-0.1534377	-0.15383516	-0.13847828	-0.13950005	-0.1331461	-0.106844254	-0.16724119	-0.1544005	-0.16359103	-0.19033054	-0.1975036	-0.17339367	-0.21132101	-0.15893744	-0.19693519	-0.1846154	-0.20703653	-0.17559814	-0.16647863	-0.18185183	-0.15825287	-0.15747382	-0.16821398	-0.14622161	-0.16204284	-0.16264492	-0.15574892	-0.12587924	-0.13298164	-0.14276657	-0.17488295	-0.15437688	-0.151522	-0.17631638	-0.15853488	-0.18351865	-0.17077407	-0.15735702	-0.17228758	-0.1638332	-0.16875863	-0.19756709	-0.15934035	-0.14337003	-0.15661393	-0.15372434	-0.15727015	-0.15722841	-0.15618971	-0.16603953	-0.13359004	-0.18432999	-0.14068794	-0.14840236	-0.1623871	-0.16054069	-0.15122296	-0.13267109	-0.14214456	-0.1756534	-0.18461828	-0.15687966	-0.15200791	-0.16091031	-0.15370715	-0.15376139	-0.13417412	-0.1601014	-0.14483562	-0.14312418	-0.1532838	-0.16473585	-0.14475769	-0.1365859	-0.13826361	-0.14274083	-0.1584372	-0.15080164	-0.15352939	-0.15122367	-0.18086414	-0.11847666	-0.16298404	-0.150288	-0.12639357	-0.17029819	-0.14150146	-0.1534052	-0.13956727	-0.13657153	-0.16417211	-0.13371393	-0.16624437	-0.15049359	-0.16011006	-0.12288412	-0.1417297	-0.14174446	-0.17893614	-0.15880124	-0.13268007	-0.16378915	-0.14863105	-0.14740565	-0.15458451	-0.1589575	-0.17894481	-0.14849173	-0.14945954	-0.15260275	-0.13245365	-0.14444906	-0.11445629	-0.15520924	-0.16106226	-0.14647725	-0.15015396	-0.15634891	-0.16150683	-0.14889127	-0.15806226	-0.15883245	-0.13353749	-0.16170888	-0.14177012	-0.1419795	-0.15287021	-0.15942663	-0.15390316	-0.1577411	-0.15551558	-0.11609258')

'-0.45877382;-0.470411;-0.4674135;-0.4736074;-0.44156438;-0.47548252;-0.44735974;-0.44824904;-0.4639969;-0.45784265;-0.45644265;-0.4638867;-0.46670336;-0.46304768;-0.48638973;-0.4515526;-0.4706145;-0.49355316;-0.4642372;-0.4766898;-0.46823722;-0.45529616;-0.4491247;-0.48889023;-0.4732201;-0.4671613;-0.4744037;-0.45908988;-0.49704504;-0.4645567;-0.47549337;-0.4682126;-0.43353945;-0.46368045;-0.44632033;-0.4935618;-0.45706254;-0.4611628;-0.46721405;-0.46532038;-0.45529568;-0.4515638;-0.43584064;-0.46726593;-0.45786634;-0.46373707;-0.45947528;-0.47260565;-0.4767009;-0.4698779;-0.4660646;-0.4894919;-0.46707913;-0.46301925;-0.4391646;-0.4473611;-0.4579038;-0.43909606;-0.46308517;-0.4789041;-0.47968763;-0.4640072;-0.46033573;-0.44862735;-0.4606386;-0.49757892;-0.48357502;-0.4733503;-0.45197883;-0.45779502;-0.4625774;-0.473577;-0.45132506;-0.46539423;-0.48512572;-0.44213033;-0.4576674;-0.44851744;-0.45456725;-0.45654044;-0.43471223;-0.47352314;-0.4519704;-0.47006404;-0.4716673;-0.48033005;-0.